In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import keras.backend as K
import scipy.sparse as sp
import random

2024-07-12 09:17:37.835866: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-12 09:17:38.068032: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 09:17:38.068078: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 09:17:38.119139: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-12 09:17:38.195413: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# load the trained model
model = tf.keras.models.load_model("..//models//dualgcn_trained_on_domain")

2024-07-12 09:17:49.723663: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0


In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 691, 1)]             0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 691, 1)]             0         []                            
                                                                                                  
 dense_2 (Dense)             (None, 691, 32)              64        ['input_3[0][0]']             
                                                                                                  
 dense_8 (Dense)             (None, 691, 32)              64        ['input_4[0][0]']             
                                                                                              

In [4]:
drug_output = model.get_layer("global_average_pooling1d").output

In [5]:
drug_output.shape

TensorShape([None, 128])

In [6]:
cnv_output = model.get_layer("global_average_pooling1d_1").output
cnv_output.shape

TensorShape([None, 256])

In [7]:
gene_expr_output = model.get_layer("global_average_pooling1d_2").output
gene_expr_output.shape

TensorShape([None, 256])

In [8]:
extract_model = tf.keras.models.Model(model.input, [cnv_output, gene_expr_output,drug_output])

In [9]:
# import the response data
drugs_cell_lines_ic50_df = pd.read_csv("..//data//drugs_cell_lines_ic50.csv") 
print(drugs_cell_lines_ic50_df.shape)
# import the drug smiles data
pubchem_drugs_smiles_df = pd.read_csv('..//data//drugs_smile_strings.csv')
print(pubchem_drugs_smiles_df.shape)

drugs_smiles_cell_lines_ic50_df = pd.merge(drugs_cell_lines_ic50_df, pubchem_drugs_smiles_df, 
                                             on = "drug_id")
drugs_smiles_cell_lines_ic50_df = drugs_smiles_cell_lines_ic50_df[["drug_id", "Cancer_Cell_Line", "Smiles", "IC50"]]
drugs_smiles_cell_lines_ic50_df.dtypes
drugs_smiles_cell_lines_ic50_df["drug_id"] = drugs_smiles_cell_lines_ic50_df["drug_id"].astype(object)


(208734, 3)
(238, 2)


In [10]:
drugs_smiles_cell_lines_ic50_df.shape

(186789, 4)

In [11]:
drugs_smiles_cell_lines_ic50_df.head()

,drug_id,Cancer_Cell_Line,Smiles,IC50
0,1001,ACH-002137,NC(=O)c1ncn(C2OC(COP(=O)(O)O)C(O)C2O)c1N,7.258918
1,1004,ACH-002137,CCC1(O)CC2CN(CCc3c([nH]c4ccccc34)C(C(=O)OC)(c3...,-3.802467
2,1005,ACH-002137,N.N.[Cl-].[Cl-].[Pt+2],4.146364
3,1006,ACH-002137,Nc1ccn(C2OC(CO)C(O)C2O)c(=O)n1,3.171367
4,1007,ACH-002137,CC(=O)OC12COC1CC(O)C1(C)C(=O)C(O)C3=C(C)C(OC(=...,-4.959442


In [12]:
import pickle

# Get drug features and adjacency information
with open("..//data//drug_gcn_features.pickle", "rb") as f:
    dict_features = pickle.load(f)

with open("..//data//drug_gcn_normalized_adj_mats.pickle", "rb") as f:
    dict_normalized_adj_mats = pickle.load(f)

In [13]:
dualgcn_train = pd.read_csv("..//data//DualGCN_Embedding_train.csv")
dualgcn_test = pd.read_csv("..//data//DualGCN_Embedding_test.csv")
pubchem_to_drugs_df = pd.read_csv('..//data//1.Drug_listMon Jun 24 09_00_55 2019.csv')
pubchem_to_drugs_df = pubchem_to_drugs_df[["drug_id", "PubCHEM"]]
pubchem_to_drugs_df["PubCHEM"] = [val if str(val).isdigit() else np.nan for val in pubchem_to_drugs_df["PubCHEM"] ]
pubchem_to_drugs_df = pubchem_to_drugs_df.dropna()
pubchem_to_drugs_df["drug_id"] = pubchem_to_drugs_df["drug_id"].astype(str)
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(drugs_smiles_cell_lines_ic50_df.drop(["IC50"], axis = 1), drugs_smiles_cell_lines_ic50_df["IC50"].values, 
                                                     test_size = 0.20, random_state = 42)

In [14]:
dualgcn_train["Drug_ID"] = dualgcn_train["Drug_ID"].astype(str)
dualgcn_test["Drug_ID"] = dualgcn_test["Drug_ID"].astype(str)
dualgcn_train = pubchem_to_drugs_df.merge(dualgcn_train, left_on = ["PubCHEM"], right_on = ["Drug_ID"])
np.mean(dualgcn_train['PubCHEM'] == dualgcn_train['Drug_ID'])
dualgcn_train = dualgcn_train[['Cell_Line', 'drug_id']]
dualgcn_test = pubchem_to_drugs_df.merge(dualgcn_test, left_on = ["PubCHEM"], right_on = ["Drug_ID"])
dualgcn_test = dualgcn_test[['Cell_Line', 'drug_id']]
x_train['drug_id'] = x_train['drug_id'].astype(str)
x_valid['drug_id'] = x_valid['drug_id'].astype(str)
x_train_valid_feats = pd.concat([x_train, x_valid], ignore_index = True)
y_train_valid = pd.concat([pd.DataFrame(y_train.reshape(-1,1)), pd.DataFrame(y_valid.reshape(-1,1))], ignore_index = True)
combo_train_valid = pd.concat([x_train_valid_feats, y_train_valid], axis = 1)
combo_train_valid.columns = ['drug_id', 'Cancer_Cell_Line', 'Smiles', 'IC50']
# filter x_train x _valid here
x_y_train = combo_train_valid.merge(dualgcn_train, left_on = ['Cancer_Cell_Line','drug_id'], right_on = [ 'Cell_Line','drug_id'])
x_y_test = combo_train_valid.merge(dualgcn_test, left_on = ['Cancer_Cell_Line','drug_id'], right_on = [ 'Cell_Line','drug_id'])

x_train, x_valid, y_train, y_valid = x_y_train.drop(["IC50", 'Cell_Line'], axis = 1), x_y_test.drop(["IC50", 'Cell_Line'], axis = 1), x_y_train["IC50"].values, x_y_test["IC50"].values

train_gcn_feats = []
train_adj_list = []
for drug_id in x_train["drug_id"].values:
    train_gcn_feats.append(dict_features[drug_id])
    train_adj_list.append(dict_normalized_adj_mats[drug_id])

valid_gcn_feats = []
valid_adj_list = []
for drug_id in x_valid["drug_id"].values:
    valid_gcn_feats.append(dict_features[drug_id])
    valid_adj_list.append(dict_normalized_adj_mats[drug_id])

train_gcn_feats = np.array(train_gcn_feats).astype("float32")
valid_gcn_feats = np.array(valid_gcn_feats).astype("float32")

train_adj_list = np.array(train_adj_list).astype("float32")
valid_adj_list = np.array(valid_adj_list).astype("float32")


In [15]:
# These models are incorrect
cancer_copy_number_model = tf.keras.models.load_model("..//models//cancer_copy_number_model_no_norm_common")
cancer_cell_gen_expr_model = tf.keras.models.load_model("..//models//cancer_cell_gen_expr_model_no_norm_common")
cancer_cell_gen_methy_model = tf.keras.models.load_model("..//models//cancer_cell_gen_methy_model_no_norm")
cancer_cell_gen_mut_model = tf.keras.models.load_model("..//models//cancer_cell_gen_mut_model_no_norm")


In [16]:
pubchem_drugs_rdkit_model = tf.keras.models.load_model("..//models//pubchem_drugs_rdkit_model_no_norm")

# cancer_copy_number_model.summary()
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
# extract drug features
drug_features_train = pubchem_drugs_rdkit_model(x_train["drug_id"].values).numpy().astype("float32")
drug_features_valid = pubchem_drugs_rdkit_model(x_valid["drug_id"].values).numpy().astype("float32")
drug_features_train = std.fit_transform(drug_features_train)
drug_features_valid = std.transform(drug_features_valid)

# extract copy number features
omics_copy_number_train = cancer_copy_number_model(x_train["Cancer_Cell_Line"].values).numpy().astype("float32")
omics_copy_number_valid = cancer_copy_number_model(x_valid["Cancer_Cell_Line"].values).numpy().astype("float32")

# extract gen expr features
omics_gen_expr_train = cancer_cell_gen_expr_model(x_train["Cancer_Cell_Line"].values).numpy().astype("float32")
omics_gen_expr_valid = cancer_cell_gen_expr_model(x_valid["Cancer_Cell_Line"].values).numpy().astype("float32")

omics_gen_copy_number_gen_expr_train = np.concatenate([np.expand_dims(omics_copy_number_train, -1),
                                                      np.expand_dims(omics_gen_expr_train, -1)], axis = -1)

omics_gen_copy_number_gen_expr_valid = np.concatenate([np.expand_dims(omics_copy_number_valid, -1),
                                                      np.expand_dims(omics_gen_expr_valid, -1)], axis = -1)

# extract gen methylation features
omics_gen_methyl_train = cancer_cell_gen_methy_model(x_train["Cancer_Cell_Line"].values).numpy().astype("float32")
omics_gen_methyl_valid = cancer_cell_gen_methy_model(x_valid["Cancer_Cell_Line"].values).numpy().astype("float32")

# extract gen mutation features
with tf.device('/cpu:0'):
    omics_gen_mut_train = cancer_cell_gen_mut_model.predict(x_train["Cancer_Cell_Line"].values, verbose = 1, batch_size = 256).astype("float32")
    omics_gen_mut_valid = cancer_cell_gen_mut_model.predict(x_valid["Cancer_Cell_Line"].values, verbose = 1, batch_size = 256).astype("float32")

smile_strings_train = x_train["Smiles"].values.reshape(-1,1)
smile_strings_valid = x_valid["Smiles"].values.reshape(-1,1)



68/68 [==============================] - 0s 5ms/step


In [17]:
selected_info_common_cell_lines = "..//data//cellline_list.txt"
selected_info_common_genes = "..//data//gene_list.txt"
PPI_file = "..//data//PPI_network.txt"

with open(selected_info_common_cell_lines) as f:
    common_cell_lines = [item.strip() for item in f.readlines()]

with open("..//data//common_genes.pickle", "rb") as f:
    common_genes = pickle.load(f)

idx_dic={}
for index, item in enumerate(common_genes):
    idx_dic[item] = index

ppi_adj_info = [[] for item in common_genes] 
for line in open(PPI_file).readlines():
    gene1,gene2 = line.split('\t')[0],line.split('\t')[1]
    if (gene1 in common_genes) & (gene2 in common_genes):
        if idx_dic[gene1]<=idx_dic[gene2]:
            ppi_adj_info[idx_dic[gene1]].append(idx_dic[gene2])
            ppi_adj_info[idx_dic[gene2]].append(idx_dic[gene1])

def CelllineGraphAdjNorm(ppi_adj_info,common_genes = common_genes):
    # with open(selected_info_common_genes) as f:
    #     common_genes = [item.strip() for item in f.readlines()]
    nb_nodes = len(common_genes)
    adj_mat = np.zeros((nb_nodes,nb_nodes),dtype='float32')
    # print(adj_mat.shape)
    for i in range(len(ppi_adj_info)):
        # print(i)
        nodes = ppi_adj_info[i]
        for each in nodes:
            adj_mat[i,each] = 1
    assert np.allclose(adj_mat,adj_mat.T)
    norm_adj = NormalizeAdj(adj_mat)
    return norm_adj 

def NormalizeAdj(adj):
    adj = adj + np.eye(adj.shape[0])
    d = sp.diags(np.power(np.array(adj.sum(1)), -0.5).flatten(), 0).toarray()
    a_norm = adj.dot(d).transpose().dot(d)
    return a_norm

ppi_adj = CelllineGraphAdjNorm(ppi_adj_info,common_genes)
ppi_adj = np.expand_dims(ppi_adj,0)

omics_gen_copy_number_gen_expr_train_new = (ppi_adj@omics_gen_copy_number_gen_expr_train)
omics_gen_copy_number_gen_expr_valid_new = (ppi_adj@omics_gen_copy_number_gen_expr_valid)

copy_number_train = omics_gen_copy_number_gen_expr_train_new[:,:,0:1]
copy_number_valid = omics_gen_copy_number_gen_expr_valid_new[:,:,0:1]

gene_expr_train = omics_gen_copy_number_gen_expr_train_new[:,:,1:2]
gene_expr_valid = omics_gen_copy_number_gen_expr_valid_new[:,:,1:2]


In [18]:
# Okay, now what all do we need? - need to extract the embeddings for all train and validation data

In [19]:
# train_gcn_feats.shape

In [20]:
# Train_all_features = extract_model.predict([train_gcn_feats[:80000,:, :], train_adj_list[:80000,:, :],copy_number_train[:80000,:, :],gene_expr_train[:80000,:,:]],
#                                           batch_size = 1024)

In [21]:
valid_all_featres = extract_model.predict([ valid_gcn_feats, valid_adj_list, copy_number_valid, gene_expr_valid], 
                                         batch_size = 1024)

17/17 [==============================] - 3s 125ms/step


In [22]:
# # feature output order
# # cnv_output, gene_expr_output,drug_output

# # Extract the embeddings separately for features
# Train_cnv = Train_all_features[0]
# Train_gene = Train_all_features[1]
# Train_drug = Train_all_features[2]

In [23]:
Valid_cnv = valid_all_featres[0]
Valid_gene = valid_all_featres[1]
Valid_drug = valid_all_featres[2]

In [24]:
Valid_gene.shape

(17316, 256)

In [25]:
# # save the extracted embeddings
# np.save('..//saved_output_data//Train_cnv_new_split_GCN.npy', Train_cnv)
# np.save('..//saved_output_data//Train_gene_new_split_GCN.npy', Train_gene)
# np.save('..//saved_output_data//Train_drug_new_split_GCN.npy', Train_drug)

np.save('..//saved_output_data//Valid_cnv_new_split_GCN.npy', Valid_cnv)
np.save('..//saved_output_data//Valid_gene_new_split_GCN.npy', Valid_gene)
np.save('..//saved_output_data//Valid_drug_new_split_GCN.npy', Valid_drug)

In [26]:
# y_train = y_train.reshape(-1,1)

In [27]:
# y_train

In [28]:
y_valid = y_valid.reshape(-1,1)

In [29]:
y_valid

array([[ 3.143044],
       [ 3.213836],
       [ 1.036346],
       ...,
       [ 1.73393 ],
       [-2.714591],
       [ 1.215302]])

In [30]:
np.mean(y_valid), np.std(y_valid)

(2.063824293081543, 2.8309603843576316)

In [31]:
(2.063824293081543, 2.8309603843576316)

(2.063824293081543, 2.8309603843576316)

In [32]:
np.save('..//saved_output_data//Valid_y.npy', y_valid)
# np.save('..//saved_output_data//Train_y.npy', y_train)